## Setting & Import

In [1]:
import sys
import os

# 현재 파일(main.ipynb)이 있는 디렉토리의 부모 디렉토리를 sys.path에 추가
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
import numpy as np

from Omok.state import *
from Omok.MCTS import *

from models.load_model import *
from robotics.ControlRobotArm import *

from vision.gird_method import *
from vision.test_code import *
# from vision.rectify import *

from utils.valid_tool import *

from config import *

In [ ]:
# State / board 
State = select_state(STATE_DIM)

# alpha zero setting
mcts = MCTS(N_PLAYOUT)
get_next_action = mcts.get_legal_actions_of(model, 0, with_policy=True)

# controller setting
controller = ControlRobotArm(target_position_path=f'{F_CWD}/{P_NAME}/robotics/final_positions.csv',
                             serial_path='/dev/cu.usbserial-110')

### 프로토콜
1. 선공, 후공을 정한다.  
2. CV에서 받은 현 상태와 이전 상태를 비교해 행해진 행동을 추적한다.  
3. 알파제로가 반환한 행동 좌표를 로봇팔을 이용해 바둑판에 업데이트한다.  
4. 2-3 과정을 게임 종료까지 반복한다. 

### Protocol
1.	Determine the first and second player.
2.	Compare the current state received from CV with the previous state to track the action taken.
3.	Use the robotic arm to update the board with the move returned by AlphaZero.
4.	Repeat steps 2-3 until the game ends.

## CV 컨트롤 버전 (Control by CV)

#### fuctions

In [ ]:
def flatten_idx(coord):
    return coord[0] * STATE_SHAPE[1] + coord[1]

In [ ]:
def abstract_action(current_state, previous_state):
    if np.all(current_state == np.zeros(shape=(STATE_SHAPE))):
        return 0
    else:
        current_state = current_state.reshape(STATE_SHAPE)
        action = np.array(range(81)).reshape(*STATE_SHAPE)[current_state != previous_state]
    return int(action)

In [ ]:
def transpose_into_cv_state(state):
    board = state.player_state + state.enemy_state * -1 if state.is_first_player() else state.player_state * -1 + state.enemy_state
    return board

#### main

In [ ]:
## CV 컨트롤 버전

# 1. 선공, 후공을 정한다. 
is_player_turn = 1
# bool(input("Wanna play first? : Press 1 when your answer is YES, 0 otherwise."))

state = State()
previous_cv_state = np.zeros(shape=STATE_SHAPE)
n_steps = 0

while True:
    if state.is_done():
        break
    
    # 2. CV에서 받은 현 상태와 이전 상태를 비교해 행해진 행동을 추적한다.  
    if is_player_turn:
        # ================== CV ======================
        image_state = take_photo()
        print(image_state)

        cropped_output = "omok3.jpg"
        crop_board_from_image(image_state, cropped_output)

        image_path = "omok3.jpg"
        cv_state, black_stones, white_stones = analyze_omok_board(image_path=image_path, 
                                                               cell_size=30,
                                                               grid_size=19,
                                                               show_grid=True,
                                                               improve_detection=True)

        print(cv_state)
        action = abstract_action(cv_state, previous_cv_state)
        # ================== CV ======================

        coord = divmod(action, STATE_SHAPE[1])
        print(f"[{n_steps}] Player's Action is : {(int(coord[0]), int(coord[1]))}")

        state = state.next(action)
        is_player_turn = False
        n_steps += 1

    # 3. 알파제로가 반환한 행동 좌표를 로봇팔을 이용해 바둑판에 업데이트한다.  
    else:
        # MCTS actions 
        action, policy, n_visits = get_next_action(state)
        coord = divmod(action, STATE_SHAPE[1])
        print(f"[{n_steps}] Alpha Zero's Action is : {(int(coord[0]), int(coord[1]))}")

        # 시각화 
        visualize_pack('best', state(), n_visits, policy, action)

        # 로봇 행동
        controller.move_to_coord(coord)

        # 업데이트 
        is_player_turn = True
        previous_cv_state = transpose_into_cv_state(state)
        state = state.next(action)
        n_steps += 1

2025-02-25 18:54:52.743 python[72627:40339909] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2025-02-25 18:54:54.106 python[72627:40339909] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-25 18:54:54.106 python[72627:40339909] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Captured: 20250225_185520_omok.jpg
20250225_185520_omok.jpg
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[0] Player's Action is : (0, 0)
[1] Alpha Zero's Action is : (3, 6)
start
(np.int64(3), np.int64(6))'s angle : [73, 126, 79, 15]


## 좌표 입력 버전 (Control by non-CV ; vs with coord)

#### main

In [ ]:
# 1. 선공, 후공을 정한다. 
is_player_turn = 0 # bool(input("Wanna play first? : Press 1 when your answer is YES, 0 otherwise."))

state = State()

while True:
    if state.is_done():
        break
    
    # 2. CV에서 받은 현 상태와 이전 상태를 비교해 행해진 행동을 추적한다.  
    if is_player_turn:
        x, y = map(int, input("Enter coordinates (x,y): ").split(','))
        print(f"My action is ({x}, {y})")
        my_action = flatten_idx((x,y))
        previous_state = copy.deepcopy(state)
        state = state.next(my_action)
        is_player_turn = False

    # 3. 알파제로가 반환한 행동 좌표를 로봇팔을 이용해 바둑판에 업데이트한다.  
    else:
        # MCTS actions 
        action, policy, n_visits = get_next_action(state)
        coord = divmod(int(action), STATE_SHAPE[1])
        print(f"Alpha Zero's Action is : {(coord[0], coord[1])}")

        # 시각화 
        visualize_pack('best', state(), n_visits, policy, action)

        # 로봇 행동
        controller.move_to_coord(coord)

        # 업데이트 
        is_player_turn = True
        previous_state = copy.deepcopy(state)
        state = state.next(action)